In [1]:
# 17_search.ipynb

%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/app')

from IPython.display import Markdown

In [2]:
from hybrid_search import hybrid_search

In [3]:
from qdrant_search import QdrantSearchEngine
from bm25_search import ElasticsearchBM25
voyage_apikey = 'pa-g4x1yHL4Hv6qBjXcrT2Nhm5gG0srlDhVBp6Op7_IJBQ'


qdrant_engine = QdrantSearchEngine(voyage_apikey)
es_engine = await ElasticsearchBM25.create() 

Connecting to Elasticsearch at elasticsearch:9200


In [4]:
q = """
meeting notetakers
"""

In [5]:
import os
QDRANT_HOST = os.getenv('QDRANT_HOST')
QDRANT_PORT = os.getenv('QDRANT_PORT')

In [6]:
from qdrant_client import AsyncQdrantClient

In [7]:
query = "software"
content_ids = None
k = 100

In [8]:
semantic_results = []
if qdrant_engine:
    semantic_results = await qdrant_engine.search(
        query=query,
        content_ids=content_ids,
        k=k
    )
text_results = []
if es_engine:
    text_results = await es_engine.search(
        query=query,
        content_ids=content_ids,
        k=k
    )

/usr/local/lib/python3.11/site-packages/voyageai/client.py:58: UserWarning: The `model` argument is not specified and defaults to voyage-2. It will be a required argument in the future. We recommend to specify the model when using this function. Please see https://docs.voyageai.com/docs/embeddings for the list of latest models provided by Voyage AI.
  warnings.warn(


In [9]:
len(semantic_results)

34

In [10]:
len(text_results)

4

In [13]:
# Merge results
all_results = []
seen_content = set()

# Process semantic results first
for result in semantic_results:
    content_id = result.payload.get('content_id')
    if content_id not in seen_content:
        seen_content.add(content_id)
        all_results.append({
            'score': result.score,
            'content': result.payload.get('content', ''),
            'content_id': content_id,
            'timestamp': result.payload.get('timestamp'),
            'formatted_time': result.payload.get('formatted_time'),
            'contextualized_content': result.payload.get('contextualized_content', ''),
            'content_type': result.payload.get('content_type', '')
            
            
            
            
        })

In [14]:
len(all_results)

2

In [15]:

# Process text results
for hit in text_results['hits']['hits']:
    content_id = hit['_source'].get('content_id')
    if content_id not in seen_content:
        seen_content.add(content_id)
        all_results.append({
            'score': hit['_score'],
            'content': hit['_source'].get('content', ''),
            'content_id': content_id,
            'timestamp': hit['_source'].get('timestamp'),
            'formatted_time': hit['_source'].get('formatted_time'),
            'contextualized_content': hit['_source'].get('contextualized_content', ''),
            'content_type': hit['_source'].get('content_type', '')
        })

# Sort by score descending
all_results.sort(key=lambda x: x['score'], reverse=True)

In [16]:
all_results

[{'score': 0.04467252,
  'content': 'Artem Puzik:  этот кусок верстаю компонент и пытаясь там опи я примерно знаю какая там взять типа meetings all и пытаюсь отрисовать эту историю вот дальше уже надо будет понятно что я могу верстать пустые вот эти вот компоненты но их надо будет скоро там чем-то заполнять но я сегодня я сегодня сделаю я понял что нужно сделать ой еще бы я дима знаешь что попросил допустим тут типа кнопка да не знаю можно фиг мини фигни фигни можно какие-то там ссылочки или пометки оставлять да и может быть Условно, тут была бы пометка, что там, раз, типа, get all, там, meetings all с такими-то параметрами.  Ну, просто в этом, в свагере, как бы, большое...  Твой список. Не буду каждый тянуть и смотреть, что он дает, чтобы понимать, что оно не оно.  Допустим, транскрипт. Тут ссылочка, что нажимаешь и должен спросить твой бэк.  о том-то, о том-то. Я спрашиваю, смотрю, ой, типа круто получил, отрисовал. Если я вижу, что оно похоже не совсем на то, что нарисовано, вот, мы

In [17]:
r = await hybrid_search(q, qdrant_engine, es_engine)

/usr/local/lib/python3.11/site-packages/voyageai/client.py:58: UserWarning: The `model` argument is not specified and defaults to voyage-2. It will be a required argument in the future. We recommend to specify the model when using this function. Please see https://docs.voyageai.com/docs/embeddings for the list of latest models provided by Voyage AI.
  warnings.warn(


In [18]:
import pandas as pd
df = pd.DataFrame(r['results'])

In [19]:
df

,score,content,content_id,timestamp,formatted_time,contextualized_content
0,0.049259,Artem Puzik: этот кусок верстаю компонент и п...,580bb613-00de-4c4a-b048-2c2576fcdef0,2025-01-23T09:40:30.720000+00:00,2025-01-23 09:40:30,The chunk is part of a conversation between Ar...
1,0.040986,Dmitriy Grankin: Или сейчас можем пойти посмо...,6a7287fb-b6ae-46cf-8b99-1ce1bcc81403,2025-01-21T15:36:56.606000+00:00,2025-01-21 15:36:56,The chunk is part of a discussion among team m...
2,0.036078,"Alex Shevliakov: Да, тем более, если ты хочеш...",fb996504-0475-4ee9-a85e-dda4e9f15a4b,2025-01-15T14:07:51.364000+00:00,2025-01-15 14:07:51,The chunk is part of a conversation between Al...
3,0.014125,Mariana Montenegro: half an hour yeah okay um...,a0c3f703-2a5e-4ed0-9561-807121516984,2025-01-21T15:02:50.184000+00:00,2025-01-21 15:02:50,This chunk is part of a conversation where Mar...


In [20]:
def format_meeting_context(df, show_content=True, show_contextualized=True):
    unique_meetings = df['content_id'].unique()
    meeting_map = {mid: idx+1 for idx, mid in enumerate(unique_meetings)}
    meeting_map_reverse = {v:k for k,v in meeting_map.items()}
    
    # Sort by timestamp for meetings, then by formatted_time within meetings
    df = df.sort_values(['timestamp', 'formatted_time'])
    
    meetings = []
    for content_id, group in df.groupby('content_id'):
        int_content_id = meeting_map[content_id]
        timestamp = pd.to_datetime(group['timestamp'].iloc[0])
        date_header = f"## Meeting {int_content_id} - {timestamp.strftime('%B %d, %Y %H:%M')}"
        
        content_items = []
        for _, row in group.iterrows():
            time_prefix = f"[{row['formatted_time']}]" if row['formatted_time'] else ''
            if show_contextualized:
                content_items.append(f"- {time_prefix} {row['contextualized_content']}")
            if show_content:
                prefix = "  > " if show_contextualized else "- "
                content_items.append(f"{prefix}{row['content']}")
        
        content = "\n".join(content_items)
        meetings.append(f"{date_header}\n\n{content}\n")
    
    return "\n".join(meetings), meeting_map_reverse

In [23]:
input_context, meeting_id_map = format_meeting_context(df,show_contextualized=True)

In [24]:
display(Markdown(input_context))

## Meeting 3 - January 23, 2025 09:40

- [2025-01-23 09:40:30] This chunk is part of a conversation between Artem Puzik and Dmitry Grankin discussing the integration of an API within an existing application, specifically focusing on user authorization and the handling of authentication tokens in the context of a chat feature.
  > Artem Puzik:  это же внутри уже готового приложения будет правильно ну то есть пользователь туда уже на эту страницу он придет авторизована или это просто что-то отдельная мне как бы окошко там авторизации я да если ты сейчас пойдешь там у тебя есть еще  можно взять пока ту страницу, которая есть, она там неадаптивная, надо будет адаптивнее справиться, но это потом.  там слово такое что сначала сначала у тебя идет google авторизация и сначала как пользователь  Нет, Дим, я о другом.  То есть, допустим, если моя страница, мой компонент или что-то там, API мое будет лежать уже внутри страницы, Твоего приложения, то есть авторизация, значит, уже пройдена.  Значит, этот токен у тебя где-то там лежит, я его найду, правильно?  Этот токен, ты его получаешь от приложения.  Женя, у тебя там как раз описано, как авторизация происходит, и ты его сохраняешь в руке.  Еще раз.  Да. Для того, чтобы дойти до этого чата, я уже у тебя в существующем Vexo App, да, уже какой-то путь прошел, правильно?  Авторизация.

## Meeting 1 - January 21, 2025 15:36

- [2025-01-21 15:36:56] The chunk is part of a discussion among Dmitriy Grankin, Artem Puzik, and Sergey Ryabenko about the user flow and functionality of an application that includes chat and meeting features. In this specific segment, Dmitriy outlines the first user flow, focusing on how users can interact with meeting cards to access transcriptions and chat related to past meetings.
  > Dmitriy Grankin:  чуть попозже сделаем вот в чем здесь ну как бы первый юзер флоу до начнем под давайте по user flow пойдем первый это Я хочу что-нибудь посмотреть по встрече, которая только прошла или какая-нибудь другая.  Я нажимаю на карт, на митинг-карт, у меня открывается...  Правая панель, в которой у меня есть транскрипция. И чат работает сейчас в контексте этой встречи. Контекст, да, это то, что мы подаем.  скажем так до чата да то есть контекст имеет разным образом определяется самый простой флоу от когда мы выбрали встречу, и он определяется по этой встрече, соответственно, по этой встрече.  Можем что-нибудь спросить про эту встречу, можем выбрать Quick Action.

## Meeting 4 - January 21, 2025 15:02

- [2025-01-21 15:02:50] This chunk is part of a conversation where Mariana Montenegro introduces herself and the AI hub at Unicorn Factory Lisboa, discussing its purpose and community focus before other participants, including Dmitry Grankin and Pedro Teixeira, join the discussion.
  > Mariana Montenegro:  half an hour yeah okay um so i think pedro is running a bit late but but well i will introduce myself firstly so my name is mariana i'm the manager the responsible for the ai hub on parts of the unicorn factory lisboa So, the AI hub is more than building with offices.

## Meeting 2 - January 15, 2025 14:07

- [2025-01-15 14:07:51] The chunk is part of a discussion between Alex Shevliakov and Dmitriy Grankin about the user experience and functionality of the VEX application, specifically focusing on the primary user scenarios for finding and interacting with calls and utilizing chat for inquiries and references.
  > Alex Shevliakov:  Ну окей, хорошо. Давай с другой стороны зайдем.  Какой основной у пользователя сценарий работы с VEX?  Наверное, два. Один – это пойти найти звонок и что-то с ним поделать по ресерче.  а другой это чат я задаю вопрос получаю какой-то ответ с референсами с референсами на звонки и могу найти эти звонки через чат то есть я попадаю нахожу звонок либо через фильтрацию и скроллинг или через чат понял но при этом как бы чат имеет основные со мной несет основную информационную нагрузку что я в этой версии даже пока не знаю надо ли реализовывать самаре какой Потому что в старой версии там есть короткое summary под каждой единицей.  Нужно ли это делать или нет?


In [10]:
[v for v in meeting_id_map.values()]

NameError: name 'meeting_id_map' is not defined

In [36]:
from core import generic_call,system_msg,user_msg,count_tokens

In [37]:
input = count_tokens(input_context)

In [38]:
input

2517

In [39]:
from core import generic_call,system_msg,user_msg


In [40]:
system_prompt = """
you are helpful assistant that is answering user requests by interpreting  while interpreting the context
You have to rely on the context to answer the user request

User is Dmitry Grankin, Address his as YOU and distinguish them in in the provided context in in the response.

Always supply reference to specific meeting with [#meeting_id]

If appropriate, inlude qoutes with speaker reference

whenever you're responding consider everything you know about me in the memory to form a context of things that I would find interesting and where possible link back to those topics and include key terminologies and concepts that will help expand my knowledge along those areas.
"""




In [41]:

r = await generic_call([system_msg(system_prompt),
                        user_msg(f'context: {input_context}'),
                        user_msg(f'request: {q}')])
r



The representatives from the AI Marketing Directory are the individuals you were speaking with during the meeting on December 12, 2024. They are involved in discussing their company's strategic pivot towards creating high-quality content specifically for founder-led businesses, which aligns with your interests in content creation and marketing strategies for Vexa. 

In the meeting, they mentioned their focus on understanding client products, analyzing market competition, and developing tailored content strategies. They also highlighted their expertise in crafting viral content and leveraging an influencer network, particularly on Twitter, which could be beneficial for your personal brand and Vexa's marketing efforts. 

If you need more specific details about their roles or the company, feel free to ask!


"The representatives from the AI Marketing Directory are the individuals you were speaking with during the meeting on December 12, 2024. They are involved in discussing their company's strategic pivot towards creating high-quality content specifically for founder-led businesses, which aligns with your interests in content creation and marketing strategies for Vexa. \n\nIn the meeting, they mentioned their focus on understanding client products, analyzing market competition, and developing tailored content strategies. They also highlighted their expertise in crafting viral content and leveraging an influencer network, particularly on Twitter, which could be beneficial for your personal brand and Vexa's marketing efforts. \n\nIf you need more specific details about their roles or the company, feel free to ask!"

In [42]:
display(Markdown(r))

The representatives from the AI Marketing Directory are the individuals you were speaking with during the meeting on December 12, 2024. They are involved in discussing their company's strategic pivot towards creating high-quality content specifically for founder-led businesses, which aligns with your interests in content creation and marketing strategies for Vexa. 

In the meeting, they mentioned their focus on understanding client products, analyzing market competition, and developing tailored content strategies. They also highlighted their expertise in crafting viral content and leveraging an influencer network, particularly on Twitter, which could be beneficial for your personal brand and Vexa's marketing efforts. 

If you need more specific details about their roles or the company, feel free to ask!

In [37]:
def extract_tagged_content(text: str, tag: str) -> str:
    import re
    # Try exact match first
    pattern = f"<{tag}>(.*?)</{tag}>"
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group(1).strip()
    
    # If no exact match, try to find content after opening tag
    pattern = f"<{tag}>(.*?)(?:</?|$)"
    match = re.search(pattern, text, re.DOTALL)
    return match.group(1).strip() if match else ""

# Example usage:
thinking = extract_tagged_content(r, "thinking")
answer = extract_tagged_content(r, "answer")

In [50]:
thinking

'Entities:\n- People: **Dmitriy Grankin**, **Lara Vargas**\n- Companies: None mentioned\n- Products: **VEXA**\n- Ideas: Engaging with **micro-influencers**, influencer outreach strategies, audience targeting, content creation, qualitative research\n- Action Points: \n  - Create a set of short descriptions about target audiences\n  - Identify and outreach to **micro-influencers**\n  - Develop a pipeline for influencer engagement\n  - Create YouTube queries for influencer research\n  - Analyze engagement rates of influencers\n  - Set up meetings with identified influencers'